In [1]:
import requests
import os
import pandas as pd
import json
import warnings
import mysql.connector
import http.client
from pickle import dumps
import http.client
import funs_gcloud as gc


from amadeus import Client, ResponseError, Location
from google.cloud import storage

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'credentials.json'
warnings.filterwarnings('ignore')

In [2]:
#Creacion objeto BDD
db = mysql.connector.connect(
    host='34.175.220.137',
    user='root',
    password='Fernando_00',
    database='demo'
)

Descarga de listado de todos los aeropuertos de la API amadeus

In [3]:
def get_eur_airports(_code):
    
    amadeus = Client(client_id='UnpIFpHb1A2LbO2l0WkYgHBqIho5Ul8n', client_secret='xflHgGy3788GCUvB')
    airports = []
    
    try:
        response = amadeus.reference_data.locations.get(
            keyword=_code,
            subType='AIRPORT',
            view='FULL',
            page={'limit': 200})
        
        while response.data:
            for airport in response.data:
                if airport['address']['regionCode'] == 'EUROP' and airport['address']['countryCode']==_code:  
                    airports.append(airport)
            
            if 'next' in response.result['meta']: response = amadeus.next(response.result['meta']['next']) 
            else: break
        return airports 

    except Exception as e:
        print(f"Error: {e}")

In [7]:
#Hay veces que devuelve un error '429', pero es debido a la saturacion de la API, volver a ejecutar el codigo
country_code = ['ES', 'FR', 'DE', 'GB', 'IT']
json_airps = []

#Como proxima mejora, añadir timer de 1 segundo entre llamadas
for country in country_code:
    json_airps.extend(get_eur_airports(country))

In [8]:
#Llamamos a archivo funs_gcloud.py para subir todos los aeropuertos al bucket
gc.upload_to_gcloud(json_airps)

File MADRID/ES:ADOLFO SUAREZ BARAJA uploaded successfully
File BARCELONA/ES:AIRPORT uploaded successfully
File PALMA DE MALLORCA/ES:PALMA DE uploaded successfully
File MALAGA/ES:MALAGA AIRPORT uploaded successfully
File GRAN CANARIA/ES:GRAN CANARIA uploaded successfully
File ALICANTE/ES:ALICANTE AIRPORT uploaded successfully
File TENERIFE/ES:TENERIFE SUR uploaded successfully
File IBIZA/ES:IBIZA uploaded successfully
File LANZAROTE/ES:LANZAROTE uploaded successfully
File VALENCIA/ES:VALENCIA AIRPORT uploaded successfully
File FUERTEVENTURA/ES:FUERTEVENTURA uploaded successfully
File TENERIFE/ES:TENERIFE NORTE uploaded successfully
File MENORCA/ES uploaded successfully
File BILBAO/ES:BILBAO AIRPORT uploaded successfully
File SEVILLA/ES:SEVILLA AIRPORT uploaded successfully
File SANTIAGO DE COMPOSTELA/ES uploaded successfully
File GIRONA/ES:COSTA BRAVA uploaded successfully
File ASTURIAS/ES:ASTURIAS uploaded successfully
File SANTA CRUZ DE LA PALMA/ES:LA P uploaded successfully
File MURC

File ROUEN/FR:VALLEE DE SEINE uploaded successfully
File VALENCE/FR:CHABEUIL uploaded successfully
File VAL-D'ISERE/FR:VAL-D'ISERE uploaded successfully
File VICHY/FR:CHARMEIL uploaded successfully
File PARIS/FR:VILLACOUBLAY VELIZY uploaded successfully
File VANNES/FR:MEUCON uploaded successfully
File VITTEL/FR:CHAMP DE COURSES uploaded successfully
File CAHORS/FR:LALBENQUE uploaded successfully
File FRANKFURT/DE:FRANKFURT INTL uploaded successfully
File MUNICH/DE:MUNICH INTERNATIONAL uploaded successfully
File BERLIN/DE:BRANDENBURG uploaded successfully
File DUESSELDORF/DE:DUESSELDORF uploaded successfully
File BERLIN/DE:TEGEL uploaded successfully
File HAMBURG/DE:HAMBURG uploaded successfully
File COLOGNE/BONN/DE:COLOGNE/BONN uploaded successfully
File STUTTGART/DE:STUTTGART AIRPORT uploaded successfully
File HANNOVER/DE:HANNOVER uploaded successfully
File NUREMBERG/DE:NUREMBERG AIRPORT uploaded successfully
File FRANKFURT/DE:HAHN AIRPORT uploaded successfully
File BREMEN/DE:BREMEN u

File BALLYKELLY/GB uploaded successfully
File LONDON/GB:BIGGIN HILL uploaded successfully
File BARROW IN FURNESS/GB uploaded successfully
File BRIZE NORTON/GB:RAF STATION uploaded successfully
File CARLISLE/GB uploaded successfully
File COLL ISLAND/GB uploaded successfully
File COLONSAY/GB uploaded successfully
File COVENTRY/GB uploaded successfully
File DORNOCH/GB uploaded successfully
File ENNISKILLEN/GB:ST ANGELO uploaded successfully
File SHOREHAM BY SEA/GB uploaded successfully
File FARNBOROUGH/GB uploaded successfully
File FETLAR/GB uploaded successfully
File FAIRFORD/GB:RAF STATION uploaded successfully
File FLOTTA/GB uploaded successfully
File FORRES/GB:KINLOSS RAF uploaded successfully
File FILTON/GB uploaded successfully
File KEMBLE/GB:COTSWOLD uploaded successfully
File HAVERFORDWEST/GB uploaded successfully
File HOY/GB:LONGHOPE uploaded successfully
File HIGH WYCOMBE/GB:AIR PARKER BOO uploaded successfully
File KING'S LYNN/GB:MARHAM RAF uploaded successfully
File BRANDON/GB

In [9]:
#Llamamos a archivo funs_gcloud.py para obtener el nombre de los archivos todos los aeropuertos
airp_list = gc.getFilesList('airports/')

In [11]:
for airp in airp_list:
   
    try:
        file = gc.getFile(airp)
        file = gc.txt_2_json(file)
        cursor = db.cursor()

        query = f"""
            INSERT INTO airports_eur VALUES (
            0,
            '{file['name'].replace("'",'' )}',
            '{file['detailedName'].replace("'",'' )}',
            '{file['id'].replace("'",'' )}',
            '{file['iataCode']}',
            {float(file['geoCode']['latitude'])},
            {float(file['geoCode']['longitude'])},
            '{file['address']['cityName'].replace("'",'' )}',
            '{file['address']['cityCode'].replace("'",'' )}',
            '{file['address']['countryCode'].replace("'",'' )}'
            )
        """   
        cursor.execute(query)
        db.commit()
        print('File {} uploaded successfully to SQL'.format(file['detailedName']))

    except Exception as e:
        print(f"Error: {e}")
    

File AACHEN/DE:MERZBRUECK uploaded successfully to SQL
File ABERDEEN/GB:DYCE uploaded successfully to SQL
File AGEN/FR:LA GARENNE uploaded successfully to SQL
File AJACCIO/FR:NAPOLEON BONAPARTE uploaded successfully to SQL
File ALBACETE/ES:AIRPORT uploaded successfully to SQL
File ALBENGA/IT uploaded successfully to SQL
File ALBERT/FR:BRAY uploaded successfully to SQL
File ALBI/FR:LE SEQUESTRE uploaded successfully to SQL
File ALDERNEY/GB uploaded successfully to SQL
File ALGHERO/IT:FERTILIA uploaded successfully to SQL
File ALICANTE/ES:ALICANTE AIRPORT uploaded successfully to SQL
File ALMERIA/ES uploaded successfully to SQL
File ALTENBURG/DE:NOBITZ uploaded successfully to SQL
File ANCONA/IT:FALCONARA uploaded successfully to SQL
File ANGERS/FR:MARCE uploaded successfully to SQL
File ANGLESEY/GB:VALLEY uploaded successfully to SQL
File ANGOULEME/FR:BRIE-CHAMPNIERS uploaded successfully to SQL
File ANNECY/FR:MEYTHET uploaded successfully to SQL
File AOSTA/IT:CORRADO GEX AIRPORT upload

File FRANKFURT/DE:HAHN AIRPORT uploaded successfully to SQL
File FRIEDRICHSHAFEN/DE:FRIEDRICHSH uploaded successfully to SQL
File FRITZLAR/DE:FRITZLAR APT uploaded successfully to SQL
File FUERSTENFELDBRUCK/DE uploaded successfully to SQL
File FUERTEVENTURA/ES:FUERTEVENTURA uploaded successfully to SQL
File GAP/FR:TALLARD uploaded successfully to SQL
File GEILENKIRCHEN/DE:AIR BASE uploaded successfully to SQL
File GENOA/IT:CRISTOFORO COLOMBO uploaded successfully to SQL
File GIEBELSTADT/DE uploaded successfully to SQL
File GIRONA/ES:COSTA BRAVA uploaded successfully to SQL
File GLASGOW/GB:GLASGOW INTL uploaded successfully to SQL
File GLASGOW/GB:PRESTWICK uploaded successfully to SQL
File GLOUCESTER CHELTENHAM/GB:GLOUC uploaded successfully to SQL
File GRANADA/ES:FEDERICO GARCIA LOR uploaded successfully to SQL
File GRANVILLE/FR uploaded successfully to SQL
File GRAN CANARIA/ES:GRAN CANARIA uploaded successfully to SQL
File GROSSETO/IT:CORRADO BACCARINI uploaded successfully to SQL
Fil

File NORDERNEY/DE uploaded successfully to SQL
File NORTHAMPTON/GB:SYWELL uploaded successfully to SQL
File NORTHOLT/GB:RAF STATION uploaded successfully to SQL
File NORTH RONALDSAY/GB uploaded successfully to SQL
File NORWICH/GB:INTERNATIONAL uploaded successfully to SQL
File NOTTINGHAM/GB:EAST MIDLANDS uploaded successfully to SQL
File NOTTINGHAM/GB:NOTTINGHAM AIRPO uploaded successfully to SQL
File NUREMBERG/DE:NUREMBERG AIRPORT uploaded successfully to SQL
File OAKHAM/GB:COTTESMORE RAF uploaded successfully to SQL
File OBAN/GB:NORTH CONNEL uploaded successfully to SQL
File OBERPFAFFENHOFEN/DE uploaded successfully to SQL
File ODIHAM/GB:RAF STATION uploaded successfully to SQL
File OLBIA/IT:COSTA SMERALDA uploaded successfully to SQL
File ORISTANO/IT:FENOSU uploaded successfully to SQL
File ORLEANS/FR:ST DENIS DE L HOTEL uploaded successfully to SQL
File OUESSANT/FR:OUESSANT uploaded successfully to SQL
File OUT SKERRIES/GB uploaded successfully to SQL
File OXFORD/GB:LONDON OXFORD u

File YEOVILTON/GB:ROYAL NAVAL AIR S uploaded successfully to SQL
File ZARAGOZA/ES:ZARAGOZA uploaded successfully to SQL


#### Parte 2
##### Crea un diagrama de relaciones (Puedes utilizar Miro, por ejemplo) indicando que tabla se relaciona con cual y que campos con cuales. Esto te ayudará a resolver la parte 3

In [12]:
dba = mysql.connector.connect(
    host='iosqlde.onairnet.xyz',
    user='vienna_airport',
    password='Vie23@#@',
    database='aeropuerto_viena'
)

def sql(_query):
    return pd.read_sql_query(_query, dba)


NotSupportedError: Authentication plugin 'caching_sha2_password' is not supported

¿Cuantos litros se han repostado en cada zona operacional? Ordénalo de menor a mayor

In [6]:
sql(f"""
    SELECT zonas_operacionales.nombre, SUM(repostajes.litros) as litros_repostados
        FROM zonas_operacionales
    INNER JOIN gasolineras ON zonas_operacionales.id = gasolineras.id_zona
    INNER JOIN repostajes ON gasolineras.id = repostajes.id_gasolinera
    GROUP BY zonas_operacionales.id
    ORDER BY litros_repostados ASC;'
            )
        """   )

NameError: name 'sql' is not defined

¿Cuántos vehículos han repostado más de 5L de RyanairHandling en la zona operacional A?¿Y en la E?


In [198]:
sql(f"""
SELECT zonas_operacionales.id, zonas_operacionales.nombre, COUNT(DISTINCT vehiculos.id) as total_vehiculos
    FROM repostajes
INNER JOIN vehiculos ON repostajes.id_vehiculo = vehiculos.id
INNER JOIN handling ON vehiculos.id_handling = handling.id
INNER JOIN gasolineras ON repostajes.id_gasolinera = gasolineras.id
INNER JOIN zonas_operacionales ON gasolineras.id_zona = zonas_operacionales.id
    WHERE handling.nombre = 'RYANAIR HANDLING' AND repostajes.litros > 5
GROUP BY zonas_operacionales.id, zonas_operacionales.nombre;
            )
        """   )

,id,nombre,total_vehiculos
0,1,ZONA A,26
1,2,ZONA B,20
2,3,ZONA C,22
3,4,ZONA D,25
4,5,ZONA E,27


¿Cuántos vehículos de media dan servicio a cada vuelo?


In [119]:
sql(f"""
SELECT AVG(num_vehiculos) as media_vehiculos_vuelo
    FROM (
        SELECT vuelos.id, COUNT(DISTINCT servicios.id_vehiculo) as num_vehiculos
        FROM vuelos
        JOIN servicios ON vuelos.id = servicios.id_vuelo
        JOIN vehiculos ON servicios.id_vehiculo = vehiculos.id
        GROUP BY vuelos.id
    ) as med;
        """   )

,media_vehiculos_vuelo
0,3.9367


¿Cuál es el mayor operador de handling cada mes por cantidad de minutos de servicio prestados?

In [146]:
sql(f"""
SELECT 
    t.id,
    t.nombre,
    t.mes,
    t.minutos_prestados
FROM (
    SELECT 
        handling.id,
        handling.nombre,
        MONTH(vuelos.fecha) as mes,
        SUM(servicios.minutos) as minutos_prestados,
        ROW_NUMBER() OVER (PARTITION BY MONTH(vuelos.fecha) ORDER BY SUM(servicios.minutos) DESC) as rn
    FROM 
        vuelos 
        JOIN servicios ON vuelos.id = servicios.id_vuelo 
        JOIN vehiculos ON servicios.id_vehiculo = vehiculos.id 
        JOIN handling ON vehiculos.id_handling = handling.id 
    GROUP BY 
        handling.id,
        handling.nombre,
        MONTH(vuelos.fecha)
) t
WHERE t.rn = 1
ORDER BY t.mes ASC;
            )
        """   )

,id,nombre,mes,minutos_prestados
0,1,VIENNA HANDLING,1,8022.0
1,1,VIENNA HANDLING,2,7458.0
2,1,VIENNA HANDLING,3,8017.0
3,5,EASYHANDLING,4,6504.0
4,1,VIENNA HANDLING,5,8940.0
5,1,VIENNA HANDLING,6,7630.0
6,1,VIENNA HANDLING,7,7728.0
7,1,VIENNA HANDLING,8,7633.0
8,1,VIENNA HANDLING,9,7611.0
9,1,VIENNA HANDLING,10,7457.0


Teniendo en cuenta que cada hora de servicio se factura a 60€, ¿Cuál es el beneficio por vehículo por cada empresa de handling?(Ten en cuenta que el combustible es un coste para ese vehículo)

In [154]:
sql(f"""
SELECT 
    handling.id AS id_handling,
    handling.nombre AS nombre_handling,
    SUM(repostajes.litros * repostajes.precio_litro) AS coste_combustible,
    (SUM(servicios.minutos) / 60) * 60 AS ingreso_servicio,
    ((SUM(servicios.minutos) / 60) * 60) - SUM(repostajes.litros * repostajes.precio_litro) AS beneficio
FROM 
    vehiculos 
    JOIN handling ON vehiculos.id_handling = handling.id 
    JOIN servicios ON vehiculos.id = servicios.id_vehiculo 
    JOIN (
        SELECT 
            id_vehiculo, 
            SUM(litros) AS litros, 
            AVG(precio_litro) AS precio_litro 
        FROM 
            repostajes 
        GROUP BY 
            id_vehiculo
    ) AS repostajes ON vehiculos.id = repostajes.id_vehiculo
GROUP BY 
    handling.id, 
    handling.nombre;
        """   )

,id_handling,nombre_handling,coste_combustible,ingreso_servicio,beneficio
0,4,RYANAIR HANDLING,200991.208006,78172.0,-122819.208006
1,2,GROUND FORCE,195474.473966,68256.0,-127218.473966
2,1,VIENNA HANDLING,261209.985067,88942.0,-172267.985067
3,5,EASYHANDLING,183529.327775,74723.0,-108806.327776
4,3,SWISSPORT,176787.083367,66616.0,-110171.083367


¿Cuánto tiempo está ocupada cada posición de media en cada zona operacional?


In [161]:
sql(f"""
SELECT zonas_operacionales.nombre AS zona_operacional, vehiculos.tipo AS tipo_vehiculo, AVG(servicios.minutos) AS tiempo_medio_ocupacion
FROM zonas_operacionales
INNER JOIN vehiculos ON zonas_operacionales.id = vehiculos.id_zona
INNER JOIN servicios ON vehiculos.id = servicios.id_vehiculo
GROUP BY zonas_operacionales.nombre, vehiculos.tipo;
        """   )

,zona_operacional,tipo_vehiculo,tiempo_medio_ocupacion
0,ZONA E,Coche de servicio,33.3098
1,ZONA E,Tractor equipaje,32.1183
2,ZONA E,Camión de repostaje,31.9087
3,ZONA E,Ambulift,32.8600
4,ZONA B,Escalera,31.5484
5,ZONA B,Cinta Maletas,34.1611
6,ZONA B,Camión de repostaje,32.2903
7,ZONA B,Ambulift,32.8817
8,ZONA B,Autobus Lanzadera,33.5617
9,ZONA B,Pushback,31.4200


¿Cuántas posiciones tiene cada zona operacional?


In [162]:
sql(f"""
SELECT zo.nombre AS zona_operacional, COUNT(*) AS num_posiciones
FROM zonas_operacionales zo
JOIN posiciones p ON zo.id = p.zone_id
GROUP BY zo.nombre;
        """   )

,zona_operacional,num_posiciones
0,ZONA A,78
1,ZONA E,74
2,ZONA B,71
3,ZONA D,64
4,ZONA C,63


¿Cuántas posiciones ha servido cada vehículo?

In [166]:
sql(f"""
SELECT v.matricula, COUNT(p.id) AS num_posiciones_servidas
FROM vehiculos v
INNER JOIN servicios s ON v.id = s.id_vehiculo
INNER JOIN vuelos vl ON s.id_vuelo = vl.id
INNER JOIN posiciones p ON vl.id_posicion = p.id
GROUP BY v.matricula;
        """   )

,matricula,num_posiciones_servidas
0,8984DSR,23
1,6915VHD,13
2,3464MYB,28
3,2078YZZ,26
4,7028ZUT,21
...,...,...
598,6616XDP,11
599,4443JCG,17
600,9760SRY,14
601,0130OCT,13


¿Cuántas puertas de embarque tiene cada terminal?


In [169]:
sql(f"""
SELECT t.nombre AS terminal, COUNT(g.id) AS num_puertas
FROM terminales t
LEFT JOIN gates g ON t.id = g.id_terminal
GROUP BY t.id;
        """   )

,terminal,num_puertas
0,Terminal 1,56
1,Terminal 2,47


¿Cuántas puertas de embarque pueden dar servicio a posiciones remotas?¿Y a posiciones con finger?¿Y a ambas?


In [175]:
sql(f"""
SELECT 
    SUM(CASE WHEN g.posicion_remota = 1 THEN 1 ELSE 0 END) AS puertas_remotas,
    SUM(CASE WHEN g.posicion_puerta = 1 THEN 1 ELSE 0 END) AS puertas_finger,
    SUM(CASE WHEN g.posicion_remota = 1 AND g.posicion_puerta = 1 THEN 1 ELSE 0 END) AS ambas
FROM 
    gates g
JOIN 
    posiciones p ON g.id_terminal = p.id;
        """   )


,puertas_remotas,puertas_finger,ambas
0,103.0,47.0,47.0


¿Cuántos tipos de posiciones tiene cada zona operacional?


In [176]:
sql(f"""
SELECT zo.nombre, COUNT(DISTINCT p.tipo) AS num_tipos
FROM zonas_operacionales zo
JOIN posiciones p ON zo.id = p.zone_id
GROUP BY zo.id;
        """   )


,nombre,num_tipos
0,ZONA A,4
1,ZONA B,4
2,ZONA C,4
3,ZONA D,4
4,ZONA E,4


¿Cuál es el ratio de vehículos de cada tipo con respecto a cada tipo de posición?


In [206]:
sql(f"""
SELECT v.tipo AS tipo_vehiculo, p.tipo AS tipo_posicion, 
       COUNT(*) / (SELECT COUNT(*) FROM vehiculos WHERE id_zona = p.zone_id) AS ratio
FROM vehiculos v
JOIN posiciones p ON v.id_zona = p.zone_id
GROUP BY v.tipo, p.tipo, p.zone_id;
        """   )


,tipo_vehiculo,tipo_posicion,ratio
0,Autobus Lanzadera,STAND S,3.0732
1,Autobus Lanzadera,STAND XL,2.7805
2,Autobus Lanzadera,STAND M,1.7561
3,Autobus Lanzadera,STAND L,3.2195
4,Coche de servicio,STAND S,3.5000
...,...,...,...
155,Autobus Lanzadera,STAND XL,1.7544
156,Ambulift,STAND XL,1.7857
157,Ambulift,STAND S,1.6071
158,Ambulift,STAND M,1.3393


¿Cuántos vuelos han tenido servico de un ambulift the swiss port?

In [210]:
sql(f"""
SELECT COUNT(DISTINCT v.id) as ambulift_swiss_port
FROM vuelos v
JOIN servicios s ON v.id = s.id_vuelo
JOIN vehiculos ve ON s.id_vehiculo = ve.id
JOIN handling h ON ve.id_handling = h.id
WHERE ve.tipo = 'ambulift' AND h.nombre = 'Swissport';
        """   )


,ambulift_swiss_port
0,210
